In [8]:
import calc_femur_and_torso_upgrade

image_path = "pros_squat/allright1.png"
actual_height_allright_input = 180

results_dict = calc_femur_and_torso_upgrade.calc_femur_and_torso_upgrade(image_path,actual_height_allright_input)

if results_dict:
    db_filename = 'pro_db.csv'

    data = {
        'Name' : ['ALLIGHT24'],
        'Height(cm)' : [actual_height_allright_input],
        'femur_length' : [results_dict['femur_length_cm']],
        'torso_length' : [results_dict['torso_length_cm']],
        'tfr_ratio' : [results_dict['tfr_ratio']],
        'scale_factor': [results_dict['scale_factor']],
        'selected_side' : [results_dict['selected_side']]
    }

    new_df = pd.DataFrame(data)

    if os.path.exists(db_filename):
        print(f"\n'{db_filename}' adding new pro's data to DB")
        existing_df = pd.read_csv(db_filename)

        if data['Name'][0] in existing_df['Name'].values:
            print(f"[Warning] '{data['Name'][0]}' is already in the DB.\nSaving Skipped!\n")
        else:
            combined_df = pd.concat([existing_df, new_df], ignore_index = True)
            combined_df.to_csv(db_filename, index = False, encoding = 'utf-8-sig')

    else:
        print(f"\n'{db_filename}' file is created.")
        new_df.to_csv(db_filename, index = False, encoding='utf-8-sig')

    print("\n-----Final DB contents-----")
    final_db = pd.read_csv(db_filename)
    print(final_db)

else:
    print("[Saving Failed] No results detected. Check the log of function!")

ALLRIGHT!!!


AttributeError: module 'mediapipe' has no attribute 'pose'